<img src="images/logos/TClogo.png" width="500">

# PRESENTS

# ROS Developers Live Class n108

<img src="img/Live-Class-108.jpg" width="" />

**REQUIREMENTS** :
- **Basics of Linux**. If you don't have that knowledge, [check our FREE online course](https://www.robotigniteacademy.com/en/course/linux-robotics/details/)
<img src="images/logos/Linux.png" width="200">

- **Python Basics**. If you don't have that knowledge, [check our FREE online course](https://www.robotigniteacademy.com/en/course/python-basics/details/)
<img src="images/logos/python.png" width="200">

### How to use this ROSject

A <a href="http://rosjects.com">**ROSject**</a> is a **ROS project** packaged in such a way that all the material it contains (**ROS code, Gazebo simulations and Notebooks**) can be shared with any body **using only a web link**. That is what we did with all the attendants to the Live Class, we shared this ROSject with them (so they can have access to all the ROS material they contain).

**Check <a heref="https://youtu.be/g2Zg31pc-XM">this Live Class video</a> to learn more about ROSjects and how to create your own ROSjects**.

You will need to have a free account at the <a href="http://rosds.online">ROS Development Studio</a> (ROSDS). Get the account and then follow the indications below.

## Let's setup the environment

### How to launch the simulation

To run the simulation we need to first set some environment variables:

In [ ]:
source /opt/ros/eloquent/setup.bash

In [ ]:
source /home/user/simulation_ws/install/setup.bash

In [ ]:
export GAZEBO_RESOURCE_PATH=/home/user/simulation_ws/src/turtlebot3_simulations/turtlebot3_gazebo:${GAZEBO_RESOURCE_PATH}

In [ ]:
export GAZEBO_MODEL_PATH=/home/user/simulation_ws/src/turtlebot3_simulations/turtlebot3_gazebo/models:${GAZEBO_MODEL_PATH}

In [ ]:
export TURTLEBOT3_MODEL=waffle

Then launch the Gazebo simulation with the following command:

In [ ]:
ros2 launch turtlebot3_gazebo empty_world.launch.py

You will have something similar to the next image in the simulation.

<img src="images/t3_sim.png" width="600"/>

<div class="bg-info text-center">
    - Notes -
</div>

In case the Gazebo simulation doesn't show, you will need to stop the **gzserver** process (you can find the process id using the tool **ps**) and execute the command again.

<br>
<div class="bg-info text-center">
    - End of Notes -
</div>

<div>
    <h2 class="text-center">
        <span class="text-primary">Live Class 108</span>
        &nbsp;
        <span class="">ROS2 Action Server</span>
    </h2>
</div>

#### This ROSject has been created by Alberto Ezquerro

### What is an Action Server?

Actions are one of the communication types in ROS 2 intended for long running tasks. They consist of three parts: a goal, a result, and feedback.

Actions are built on topics and services. Their functionality is similar to services, except actions are preemptable (you can cancel them while executing). They also provide steady feedback, as opposed to services which return a single response.

Actions use a client-server model, similar to the publisher-subscriber model (described in the topics tutorial). The “action client” node sends a goal to an “action server” node that acknowledges the goal and returns a stream of feedback and a result.

<div>
    <h2 class="text-center">
        <span class="text-primary">1.1</span>
        &nbsp;
        <span class="">Write the server code</span>
    </h2>
</div>

First of all, let's source ROS2 on our Shell, so that we can use the ROS2 command line tools.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell</p>
</th>
</tr>
</table>
<br>

In [ ]:
source /opt/ros/eloquent/setup.bash

Now, go to the **ros2_ws/src** folder in your webshell.<br><br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell</p>
</th>
</tr>
</table>

In [ ]:
cd ~/ros2_ws/src

Next let's create a Python package:<br>

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell</p>
</th>
</tr>
</table>

In [ ]:
ros2 pkg create my_action_server --build-type ament_python --dependencies rclpy

This will create inside our "src" directory a new package with some files in it.

Create a Python script in the **src** directory in <span style="color:green;">my_action_server</span>. For this exercise, just copy this simple Python code **action_server.py** shown below.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p style="background: #407EAF; color:white">action_server.py</p>
</th>
</tr>
</table>

In [ ]:
import rclpy
from rclpy.action import ActionServer
from rclpy.node import Node
from test_action.action import Test
from geometry_msgs.msg import Twist
import time

class TestActionServer(Node):

    def __init__(self):
        super().__init__('test_action_server')
        self._action_server = ActionServer(
            self,
            Test,
            'test',
            self.execute_callback)
        self.publisher_ = self.create_publisher(Twist, '/cmd_vel', 10)
    
    def execute_callback(self, goal_handle):
        self.get_logger().info('Executing goal...')

        msg = Twist()
        msg.linear.x = 0.5

        for i in range(1, goal_handle.request.secs):
            self.publisher_.publish(msg)
            time.sleep(1)

        goal_handle.succeed()

        msg.linear.x = 0.0
        self.publisher_.publish(msg)

        result = Test.Result()
        result.status = "Finished action server. Moved during %d seconds" % goal_handle.request.secs
        return result


def main(args=None):
    rclpy.init(args=args)

    test_action_server = TestActionServer()

    rclpy.spin(test_action_server)


if __name__ == '__main__':
    main()

Modify the **setup.py** file to generate an executable from the Python file you have just created.

Add the following line within the *console_scripts* brackets of the *entry_points* field:

In [ ]:
entry_points={
        'console_scripts': [
                'action_server_node = my_action_server.action_server:main',
        ],
},

Compile your package and source the workspace.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
cd ~/ros2_ws
colcon build
source ~/ros2_ws/install/setup.bash

Finally, execute the roslaunch command in the WebShell to launch your program.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
ros2 run my_action_server action_server_node

<div>
    <h2 class="text-center">
        <span class="text-primary">1.2</span>
        &nbsp;
        <span class="">Get data from the Action Server</span>
    </h2>
</div>

You can get a list of the current available action servers with the following command:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell</p>
</th>
</tr>
</table>
<br>

In [ ]:
ros2 action list

You will get an output similar to this in the Web Shell:

In [ ]:
/my_action

You can get data from an specific action with the following command:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell</p>
</th>
</tr>
</table>

In [ ]:
ros2 action info /my_action

You will get an output similar to this in the Web Shell:

In [ ]:
Action: /my_action
Action clients: 0
Action servers: 1
    /my_action_server

You can get data from the action message used with the following command:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell</p>
</th>
</tr>
</table>

In [ ]:
ros2 interface show test_action/action/Test

You will get an output similar to this in the Web Shell:

In [ ]:
int32 secs
---
string status
---
string feedback

<div>
    <h2 class="text-center">
        <span class="text-primary">1.3</span>
        &nbsp;
        <span class="">Send a goal to the Action Server</span>
    </h2>
</div>

In order to send a goal to the Action Server, you can use the following command:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell</p>
</th>
</tr>
</table>

In [ ]:
ros2 action send_goal my_action test_action/action/Test "{secs: 5}"

You will get an output similar to this:

In [ ]:
Sending goal:
     secs: 5

Goal accepted with ID: 8f2ac02684664a07af452ab1cc660084

Result:
    status: Finished action server. Moved during 5 seconds

Goal finished with status: SUCCEEDED

<div>
    <h2 class="text-center">
        <span class="text-primary">1.4</span>
        &nbsp;
        <span class="">Adding feedback to the Action Server</span>
    </h2>
</div>

In order to add feedback messages to the Action Server, you will need to add the below lines to the server code:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Add to action_server.py</p>
</th>
</tr>
</table>

In [ ]:
# Define and fill the feedback message
feedback_msg = Test.Feedback()
feedback_msg.feedback = "Robot moving forward..."

# Publish the feedback
self.get_logger().info('Feedback: {0}'.format(feedback_msg.feedback))
goal_handle.publish_feedback(feedback_msg)

Now, in order to call the Action Server with the feedback activated, you can use the following command:

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell</p>
</th>
</tr>
</table>
<br>

In [ ]:
ros2 action send_goal --feedback my_action test_action/action/Test "{secs: 5}"

You will get something like this:

In [ ]:
Waiting for an action server to become available...
Sending goal:
     secs: 5

Goal accepted with ID: dabdb24a950a46bc8b5fb26aed1c4ee7

Feedback:
    feedback: Robot moving forward...

Feedback:
    feedback: Robot moving forward...

Feedback:
    feedback: Robot moving forward...

Feedback:
    feedback: Robot moving forward...

Result:
    status: Finished action server. Robot moved during 5 seconds

Goal finished with status: SUCCEEDED

<div class="bg-info text-center">
    - Notes -
</div>

In case this command doesn't work, execute the following command in the Shell.

`$ export PYTHONPATH=/opt/ros/eloquent/lib/python3.6/site-packages:$PYTHONPATH`

<br>

You can use the below command to stop the robot's movement.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
ros2 topic pub --once /cmd_vel geometry_msgs/msg/Twist "{linear: {x: 0.0, y: 0.0, z: 0.0}, angular: {x: 0.0, y: 0.0, z: 0.0}}"

You can use the below command to stop the reset the robot's position.

<table style="float:left;background: #407EAF">
<tr>
<th>
<p class="transparent">Execute in WebShell #1</p>
</th>
</tr>
</table>

In [ ]:
ros2 service call /reset_world std_srvs/Empty

<div class="bg-info text-center">
    - End of Notes -
</div>

### GOOD JOB!

## Mission  completed!!

# If you liked this video, please support us!
# Really... we need your support!!!!

# How can you support us?
## 1. Subscribe to our ROS online academy and become a Master of ROS Development

Go to our online academy. There is no faster way and funnier to learn ROS because we use the same
method we did here.

**We call the 30/70 method**


* **30% of the time learning theory**
* **70% of the time practicing with simulated robots**

<img src="images/logos/somecourses.png">

### Check it out at http://robotignite.academy

## 2. Buy one ROS Developers T-shirt or one of our mugs!

<img src="images/logos/mugs.jpeg">

<img src="images/logos/T-shirts.png">

You can buy them at our Teespring area (https://teespring.com/stores/ros-developers)

## 3. Give us a like in Youtube and subscribe to the channel

* **Go to our Youtube Channel (https://www.youtube.com/channel/UCt6Lag-vv25fTX3e11mVY1Q) and subscribe (it is free!!!)**
* **Give us a like to this video**

# KEEP PUSHING YOUR ROS LEARNING WITH PATIENCE AND GOOD HUMOUR!

# Build the future, Become a ROS DEVELOPER